In [2]:
import numpy as np
import os
import pandas as pd
from pathlib import Path

## pls excuse
import os, sys
up1 = os.path.abspath('..'); sys.path.insert(0, up1) 
## pls excuse
import model.utilities as mu

In [3]:
# Loading features and calculating running averages
allPatientsFeaturized = Path(os.getcwd()).parent / 'data' / 'assets' / 'featurized_all_5min.csv'
apfDF: pd.DataFrame = pd.read_csv(
  allPatientsFeaturized,
  parse_dates=['time', 'basetime'],
  dtype={'patient_id': str})
apfDF

,patient_id,time,basetime,b2b_var,b2b_iqr,b2b_range,b2b_std,ecg_rate_mean,hrv_rmssd,hrv_sdnn,...,hrv_hf,hrv_lfhf,hrv_sd1,hrv_sd2,hrv_sd1sd2,hrv_pnn20,hrv_pnn50,hrv_sampen,hrv_shanen,hrv_apen
0,00,2003-01-31 09:35:00,2003-01-31 09:30:00,0.069295,3.750916,123.220947,8.257661,75.807506,5142.511932,5156.175321,...,0.000039,4.846691,3636.261725,3667.263421,0.991546,34.700315,1.577287,-0.000000,2.463452,0.000058
1,00,2003-01-31 09:40:00,2003-01-31 09:30:00,0.025064,3.750916,24.216216,3.007496,119.952219,23.178708,12.745210,...,0.006891,0.375018,16.403542,7.392565,2.218924,44.648829,0.668896,0.400656,2.264696,0.524016
2,00,2003-01-31 09:45:00,2003-01-31 09:30:00,0.278983,48.995215,71.044496,23.049572,77.621641,43.538127,170.461595,...,0.001036,1.407742,30.814854,238.815276,0.129032,43.669251,8.527132,0.226236,4.457817,0.283402
3,00,2003-01-31 09:50:00,2003-01-31 09:30:00,0.028220,2.364532,13.088784,1.904059,67.430340,36.896575,25.004615,...,0.018443,0.402011,26.128763,23.875184,1.094390,57.440476,18.750000,2.284421,3.663204,0.938840
4,00,2003-01-31 09:55:00,2003-01-31 09:30:00,0.074071,3.253393,58.298884,5.146241,69.193872,58.292700,59.109475,...,0.049523,1.261990,41.278575,72.550158,0.568966,56.395349,23.837209,1.202086,4.206069,1.049558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22910,75,2003-07-19 17:11:39,2003-07-18 20:36:39,0.199863,23.547170,84.111906,16.849522,81.701444,207.555924,151.049052,...,0.078372,0.428181,146.945553,154.691063,0.949929,91.089109,77.970297,2.222212,6.054543,1.191244
22911,75,2003-07-19 17:16:39,2003-07-18 20:36:39,0.202433,23.039114,80.883436,16.748679,80.092740,220.785714,160.173888,...,0.083934,0.489668,156.317413,163.592907,0.955527,91.392405,80.759494,1.976393,6.128631,1.284518
22912,75,2003-07-19 17:21:39,2003-07-18 20:36:39,0.197343,22.361985,77.803922,16.532443,81.321261,205.742031,147.498688,...,0.095118,0.311996,145.663300,149.548723,0.974019,89.054726,78.606965,2.195000,6.011831,1.204267
22913,75,2003-07-19 17:26:39,2003-07-18 20:36:39,0.243248,27.909774,109.327286,21.586609,84.675915,207.615012,169.007125,...,0.025278,0.279250,146.982317,188.402309,0.780151,88.995215,75.837321,2.072797,6.274517,1.311130


In [4]:
# replacing featurized data frame with running sum dataframe
runningAverage = lambda npArr: np.cumsum(npArr) / np.array([i+1 for i in range(len(npArr))])
def getRunAvgs(df: pd.DataFrame):
    allFeats = mu.getModelConfig().features_nk
    resultingDF = list()
    for patientID, patientsFeats in df.groupby('patient_id'):
        #for each patientFeat we're gonna calc the running average at each point in time
        patientsFeats.sort_values(by='time', inplace=True)
        for feat in allFeats:
            patientsFeats[feat] = runningAverage(patientsFeats[feat].to_numpy())
        resultingDF.append(patientsFeats)
    return pd.concat(resultingDF)
raDF = getRunAvgs(apfDF)
raDF.reset_index(inplace=True)
raDF

,index,patient_id,time,basetime,b2b_var,b2b_iqr,b2b_range,b2b_std,ecg_rate_mean,hrv_rmssd,...,hrv_hf,hrv_lfhf,hrv_sd1,hrv_sd2,hrv_sd1sd2,hrv_pnn20,hrv_pnn50,hrv_sampen,hrv_shanen,hrv_apen
0,0,00,2003-01-31 09:35:00,2003-01-31 09:30:00,0.069295,3.750916,123.220947,8.257661,75.807506,5142.511932,...,0.000039,4.846691,3636.261725,3667.263421,0.991546,34.700315,1.577287,-0.000000,2.463452,0.000058
1,1,00,2003-01-31 09:40:00,2003-01-31 09:30:00,0.047179,3.750916,73.718581,5.632578,97.879863,2582.845320,...,0.003465,2.610854,1826.332634,1837.327993,1.605235,39.674572,1.123092,0.200328,2.364074,0.262037
2,2,00,2003-01-31 09:45:00,2003-01-31 09:30:00,0.124447,18.832349,72.827220,11.438243,91.127122,1736.409589,...,0.002655,2.209817,1227.826707,1304.490421,1.113168,41.006132,3.591105,0.208964,3.061988,0.269159
3,3,00,2003-01-31 09:50:00,2003-01-31 09:30:00,0.100391,14.715395,57.892611,9.054697,85.202926,1311.531336,...,0.006602,1.757865,927.402221,984.336611,1.108473,45.114718,7.380829,0.727828,3.212292,0.436579
4,4,00,2003-01-31 09:55:00,2003-01-31 09:30:00,0.095127,12.422994,57.973866,8.273006,82.001116,1060.883609,...,0.015186,1.658690,750.177492,801.979321,1.000572,47.370844,10.672105,0.822680,3.411048,0.559175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22910,22910,75,2003-07-19 17:11:39,2003-07-18 20:36:39,0.202450,32.284086,103.815525,21.560984,103.349797,167.720035,...,0.069378,0.440151,118.720146,128.856209,0.918888,87.577982,72.394471,2.103714,5.762569,1.372336
22911,22911,75,2003-07-19 17:16:39,2003-07-18 20:36:39,0.202450,32.246808,103.723057,21.541579,103.256018,167.934010,...,0.069437,0.440351,118.871748,128.996276,0.919036,87.593363,72.428201,2.103200,5.764045,1.371982
22912,22912,75,2003-07-19 17:21:39,2003-07-18 20:36:39,0.202430,32.207110,103.618964,21.521462,103.167927,168.085849,...,0.069540,0.439835,118.979345,129.078816,0.919257,87.599232,72.453015,2.103569,5.765040,1.371309
22913,22913,75,2003-07-19 17:26:39,2003-07-18 20:36:39,0.202593,32.189920,103.641797,21.521723,103.093959,168.243966,...,0.069363,0.439193,119.091356,129.316110,0.918700,87.604816,72.466553,2.103446,5.767078,1.371068


In [51]:
def getDiffFromRunningAvg(runningAverageReference, column, t, value):
    #find the nearest time that's before the point at time t
    idx = np.searchsorted(runningAverageReference['time'], t)
    return value - runningAverageReference[column].iloc[idx-1]

from prediction.europace.loadData import getAllFeatsGivenMode
def modifyFeaturizationForPatient(df: pd.DataFrame):
    allFeats = getAllFeatsGivenMode('overlapping', filterCurrent=True)#mu.getModelConfig().features_nk
    moddedDF = list()
    for patientID, patientFeats in df.groupby('patient_id'):
        patientFeats.sort_values(by='time', inplace=True)
        patientFeats.reset_index(inplace=True)

        #idea here is to go through patient's featurization and for each point in time modify it to the difference from the mean,
        # as calculated from the most recent accessible point in time
        for feat in allFeats:
            scaledToPatientValues = list()
            for time, value in zip(patientFeats['time'], patientFeats[feat]):
                diffedFromAvg = getDiffFromRunningAvg(raDF[raDF['patient_id'] == patientID], feat[:-3], time, value)
                scaledToPatientValues.append(diffedFromAvg)
            patientFeats[feat] = scaledToPatientValues
        moddedDF.append(patientFeats)
    return pd.concat(moddedDF)
res = modifyFeaturizationForPatient(pd.read_csv(
    Path(os.getcwd()).parent / 'data' / 'assets' / 'featurized_afib_predictor_data_overlapping.csv',
    parse_dates=['time', 'basetime'],
    dtype={'patient_id': str})
)



In [54]:
res.to_csv(
  Path(os.getcwd()).parent / 'data' / 'assets' / 'modded_featurization_overlapping.csv')

In [59]:
from train import trainBinaryRFC
trainBinaryRFC(overlapping=True)
trainBinaryRFC(src='../data/assets/modded_featurization.csv', overlapping=True)

________________________________________________________________________________
[Memory] Calling prediction.europace.loadData.loadDataInSurvivalFormat...
loadDataInSurvivalFormat(lengthened=False, acute=False, filterCurrent=True, overlapping=True, inDFForm=True)
124


100%|██████████| 124/124 [00:00<00:00, 2973.58it/s]

Duration since start: 0 days 00:08:29.500000
Duration since start: 0 days 00:00:34.312500
Duration since start: 0 days 00:00:41.617188
Duration since start: 0 days 00:01:45.828125
Duration since start: 0 days 01:14:41.750000
Duration since start: 0 days 00:10:48.578125
Duration since start: 0 days 00:30:46.835938
Duration since start: 0 days 00:04:31.093750
Duration since start: 0 days 02:02:24.984375
Duration since start: 0 days 00:00:59.031250
Duration since start: 0 days 00:13:59.656250
Duration since start: 0 days 01:34:52.164062
Duration since start: 0 days 00:01:04.984375
Duration since start: 0 days 00:13:51.281250
Duration since start: 0 days 00:00:20.718750
Duration since start: 0 days 01:16:10.281250
Duration since start: 0 days 00:05:07.296875
Duration since start: 0 days 00:00:49.546875
Duration since start: 0 days 00:00:43.562500
Duration since start: 0 days 00:00:47.539062
Duration since start: 0 days 00:00:55.476562
Duration since start: 0 days 01:57:18.835938
Duration s


Traceback (most recent call last):
  File "/home/rkaufman/workspace/afib_detection/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2324865/238842893.py", line 2, in <module>
    trainBinaryRFC(overlapping=True)
  File "/home/rkaufman/workspace/afib_detection/train.py", line 553, in trainBinaryRFC
    a, dfOG = loadData(lengthened=lengthened,acute=acute,filterCurrent=True, overlapping=overlapping,inDFForm=True)
  File "/home/rkaufman/workspace/afib_detection/env/lib/python3.8/site-packages/joblib/memory.py", line 594, in __call__
    return self._cached_call(args, kwargs)[0]
  File "/home/rkaufman/workspace/afib_detection/env/lib/python3.8/site-packages/joblib/memory.py", line 537, in _cached_call
    out, metadata = self.call(*args, **kwargs)
  File "/home/rkaufman/workspace/afib_detection/env/lib/python3.8/site-packages/joblib/memory.py", line 779, in call
    output